In [ ]:
%pip install foolbox advertorch

In [ ]:
import os
import foolbox as fb
from tqdm import tqdm

import torch
import torchvision
import torchvision.transforms as transforms

In [ ]:
use_cuda = True
device = torch.device("cuda" if use_cuda else "cpu")

## Test dataset attack


In [ ]:
dataset_path = "/content/test"

# Transformations for the dataset
transform = transforms.Compose([transforms.Resize((224, 224)), transforms.ToTensor()])

# Load your dataset using ImageFolder (assuming dataset is structured by classes)

kaggle_dataset = torchvision.datasets.ImageFolder(dataset_path, transform=transform)
dataloader = torch.utils.data.DataLoader(
    kaggle_dataset, batch_size=32, shuffle=False, num_workers=4, prefetch_factor=2
)

In [ ]:
model = torchvision.models.resnet50(pretrained=True).to(device).eval()
model.half()  # Convert model weights to FP16
fmodel = fb.PyTorchModel(model, bounds=(0, 1))

In [ ]:
attack = fb.attacks.L2ProjectedGradientDescentAttack(steps=10)

# Define the path to save adversarial images
adversarial_dataset_path = "adversarial_dataset"
os.makedirs(adversarial_dataset_path, exist_ok=True)

In [ ]:
epsilons = 1.0

for i, (inputs, labels) in tqdm(enumerate(dataloader), total=len(dataloader), desc="Adversarial Generation"):
    # Move inputs and labels to device
    inputs = inputs.to(device).half()  # Convert inputs to FP16
    labels = labels.to(device)

    # Generate adversarial examples using Foolbox
    with torch.cuda.amp.autocast():  # Enable mixed precision
        raw_advs, clipped_advs, success = attack(fmodel, inputs, labels, epsilons=epsilons)

    # Save adversarial images batch-wise
    for idx in range(inputs.size(0)):
        class_name = kaggle_dataset.classes[labels[idx].item()]
        class_dir = os.path.join(adversarial_dataset_path, class_name)
        os.makedirs(class_dir, exist_ok=True)

        # Save individual adversarial image
        image_path = os.path.join(class_dir, f"adversarial_{i * 16 + idx}.png")
        torchvision.utils.save_image(clipped_advs[idx].float(), image_path)  # Convert back to float for saving

print(f"Adversarial images saved to {adversarial_dataset_path}")

In [ ]:
import os
from PIL import Image
from torchvision import transforms
from tqdm import tqdm

# Path to already saved adversarial images
saved_adversarial_path = "/content/adversarial_dataset"  # Path to the original adversarial dataset
resized_adversarial_path = "./adversarial_dataset_32x32"  # Path to save resized images
os.makedirs(resized_adversarial_path, exist_ok=True)

# Define the resize transform
resize_to_32 = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor()
])

# Iterate through all saved adversarial images
for root, dirs, files in os.walk(saved_adversarial_path):
    for class_name in tqdm(dirs, desc="Processing classes"):
        class_input_path = os.path.join(root, class_name)
        class_output_path = os.path.join(resized_adversarial_path, class_name)
        os.makedirs(class_output_path, exist_ok=True)

        for image_name in tqdm(os.listdir(class_input_path), desc=f"Resizing {class_name}", leave=False):
            image_input_path = os.path.join(class_input_path, image_name)
            image_output_path = os.path.join(class_output_path, image_name)

            # Open and resize the image
            try:
                with Image.open(image_input_path) as img:
                    img = resize_to_32(img)
                    torchvision.utils.save_image(img, image_output_path)
            except Exception as e:
                print(f"Failed to process {image_input_path}: {e}")


In [ ]:
import os
from tqdm import tqdm

# Path to the dataset folder
dataset_path = "/content/adversarial_dataset_inf_test"  # Change to your folder path

# Initialize counters
total_images = 0
class_image_count = {}

# Traverse the directory
for root, dirs, files in os.walk(dataset_path):
    for class_name in tqdm(dirs, desc="Counting images by class"):
        class_path = os.path.join(root, class_name)
        image_count = len([f for f in os.listdir(class_path) if os.path.isfile(os.path.join(class_path, f))])
        class_image_count[class_name] = image_count
        total_images += image_count

# Print results
print("\n--- Image Count by Class ---")
for class_name, count in class_image_count.items():
    print(f"{class_name}: {count} images")

print(f"\nTotal images: {total_images}")


In [ ]:
import shutil
from google.colab import files

# Path to the folder in Colab
folder_to_download = "./adversarial_dataset_32x32"  # Replace with your folder path
output_zip = "./adversarial_dataset_32x32.zip"

# Compress the folder
shutil.make_archive(output_zip.replace(".zip", ""), 'zip', folder_to_download)

# Download the zipped folder
files.download(output_zip)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Attack on Training data

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Dataset path
dataset_path = "/content/train"
transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor()
])

In [ ]:
kaggle_dataset = torchvision.datasets.ImageFolder(dataset_path, transform=transform)
dataloader = torch.utils.data.DataLoader(kaggle_dataset, batch_size=8, shuffle=False, num_workers=4, prefetch_factor=2)

# Load pre-trained ResNet50
model = torchvision.models.resnet50(pretrained=True).to(device).eval()
model.half()  # Mixed precision
fmodel = fb.PyTorchModel(model, bounds=(0, 1))

In [ ]:
attack = fb.attacks.L2ProjectedGradientDescentAttack(steps=10)
epsilons = 1.0

# Transform adversarial images back to 32x32
resize_to_32 = transforms.Resize((32, 32))

In [ ]:
import os
import torch
import torchvision
from torchvision import transforms
import foolbox as fb
from tqdm import tqdm

In [ ]:
adversarial_dataset_path = "./adversarial_dataset_32x32"
os.makedirs(adversarial_dataset_path, exist_ok=True)

# Process dataset
for i, (inputs, labels) in tqdm(
    enumerate(dataloader), total=len(dataloader), desc="Adversarial Generation"
):
    try:
        # Move inputs to device
        inputs = inputs.to(device).half()
        labels = labels.to(device)

        # Generate adversarial examples
        with torch.cuda.amp.autocast():
            raw_advs, clipped_advs, success = attack(
                fmodel, inputs, labels, epsilons=epsilons
            )

        # Save images resized to 32x32 with the same filenames as originals, prefixed with "adversarial_"
        for idx in range(inputs.size(0)):
            # Get the original filename
            original_path, _ = kaggle_dataset.samples[i * inputs.size(0) + idx]
            original_filename = os.path.basename(original_path)
            adversarial_filename = f"adversarial_{original_filename}"

            # Create class directory in the output folder
            class_name = kaggle_dataset.classes[labels[idx].item()]
            class_dir = os.path.join(adversarial_dataset_path, class_name)
            os.makedirs(class_dir, exist_ok=True)

            # Resize adversarial image to 32x32 and save
            adversarial_32x32 = resize_to_32(clipped_advs[idx].float().cpu())
            image_path = os.path.join(class_dir, adversarial_filename)
            torchvision.utils.save_image(adversarial_32x32, image_path)
    except Exception as e:
        print(f"Error during batch {i + 1}: {e}")

## Linf attack


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Dataset path
dataset_path = "/content/test"
transform = transforms.Compose([transforms.Resize((32, 32)), transforms.ToTensor()])

In [ ]:
kaggle_dataset = torchvision.datasets.ImageFolder(dataset_path, transform=transform)
dataloader = torch.utils.data.DataLoader(
    kaggle_dataset, batch_size=8, shuffle=False, num_workers=4, prefetch_factor=2
)

# Load pre-trained ResNet50
model = torchvision.models.resnet50(pretrained=True).to(device).eval()
model.half()  # Mixed precision
fmodel = fb.PyTorchModel(model, bounds=(0, 1))

In [ ]:
attack = fb.attacks.LinfProjectedGradientDescentAttack(steps=10, rel_stepsize=0.1)
epsilons = 16 / 255
resize_to_32 = transforms.Resize((32, 32))

In [ ]:
adversarial_dataset_path = "./adversarial_dataset_inf_test"
os.makedirs(adversarial_dataset_path, exist_ok=True)

# Process dataset
for i, (inputs, labels) in tqdm(
    enumerate(dataloader), total=len(dataloader), desc="Adversarial Generation"
):
    try:
        inputs = inputs.to(device).half()
        labels = labels.to(device)

        # Generate adversarial examples
        with torch.cuda.amp.autocast():
            raw_advs, clipped_advs, success = attack(
                fmodel, inputs, labels, epsilons=epsilons
            )

        for idx in range(inputs.size(0)):
            # Get the original filename
            original_path, _ = kaggle_dataset.samples[i * inputs.size(0) + idx]
            original_filename = os.path.basename(original_path)
            adversarial_filename = f"adversarial_{original_filename}"

            # Create class directory in the output folder
            class_name = kaggle_dataset.classes[labels[idx].item()]
            class_dir = os.path.join(adversarial_dataset_path, class_name)
            os.makedirs(class_dir, exist_ok=True)

            adversarial_32x32 = resize_to_32(clipped_advs[idx].float().cpu())
            image_path = os.path.join(class_dir, adversarial_filename)
            torchvision.utils.save_image(adversarial_32x32, image_path)

    except Exception as e:
        print(f"Error during batch {i + 1}: {e}")